## Configurações

In [0]:
%pip install faker

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.window import Window
import pandas as pd
import numpy as np
import uuid
import random
from faker import Faker
from datetime import datetime, timedelta, date
from typing import Iterator, Tuple
import calendar
import functools

## Parâmetros de Configuração

- **NOME_APLICACAO_SPARK**: Nome identificador desta execução no ambiente Spark.
- **ANO_ESTATISTICA**: Ano base utilizado para buscar as estatísticas de volume de transações.
- **FATOR_ESCALA_VOLUME**: Fator de escala para reduzir o volume total de transações geradas. Por exemplo, `0.006` indica que será gerado apenas 0,6% do volume real, útil para criar amostras menores e mais rápidas de processar.
- **LIMITE_ABSOLUTO_TX**: Limite máximo absoluto de transações a serem geradas, evitando volumes excessivos.
- **LIMITE_MUNICIPIOS_PROCESSADOS**: Quantidade máxima de municípios para os quais serão gerados dados, priorizando aqueles com maior volume.
- **PROBABILIDADE_TRANSACAO_INTERMUNICIPAL**: Proporção de transações que terão como destino um município diferente do de origem (ex: 20%), tornando a simulação mais realista.

---

### Parâmetros de Fraude

- **PROBABILIDADE_FRAUDE_BASE**: Probabilidade padrão de uma transação ser fraudulenta (ex: 5%).
- **PROB_CONTA_ALTO_RISCO**: Probabilidade de uma conta ser classificada como "alto risco" ao ser criada (ex: 10%).
- **PROBABILIDADE_FRAUDE_CONTA_DESTINO_RISCO**: Probabilidade de fraude quando o destino é uma conta de alto risco (ex: 60%).
- **PROBABILIDADE_FRAUDE_CHAVE_RECENTE**: Probabilidade de fraude quando o PIX é destinado a uma chave recém-cadastrada (ex: 40%), já que contas novas são frequentemente usadas em golpes.
- **DIAS_CHAVE_CONSIDERADA_RECENTE**: Número de dias para considerar uma chave como "recente" (ex: 7 dias).
- **MAX_DIAS_CADASTRO_CHAVE_RISCO**: Prazo máximo para contas de alto risco cadastrarem suas chaves PIX após a abertura (ex: 5 dias).

---

### Parâmetros Gerais

- **MULTIPLICADOR_MAGNITUDE_OUTLIER**: Multiplicador para definir o valor de uma transação "outlier" (ex: 25 vezes o valor base).
- **MULTIPLICADOR_MAGNITUDE_FRAUDE**: Multiplicador para definir o valor de uma transação fraudulenta (ex: 50 vezes o valor base), simulando tentativas de golpes de alto valor.
- **PROBABILIDADES_TIPO_FRAUDE**: Dicionário que define a chance de cada tipo de fraude ocorrer. Exemplo: `valor_atipico` tem 30% de chance de ser o tipo escolhido em caso de fraude.
- **PESO_CONTAS_POS_PIX**: Proporção de contas criadas após o lançamento do PIX (ex: 70%), refletindo o aumento da bancarização no período.

In [0]:

# -----------------------------------------------------------------------------
# 2. PARÂMETROS GLOBAIS DE CONFIGURAÇÃO
# -----------------------------------------------------------------------------
print("INFO: Definindo parâmetros globais...")
NOME_APLICACAO_SPARK = "GeradorDadosPix_Absoluto_v10.23_LogicaAvancada"

# === PARÂMETROS DE ESCALA E VOLUME ===
FATOR_ESCALA_VOLUME = 0.3 # Ajustado para 1% do volume estatístico base
LIMITE_ABSOLUTO_TX = 20000 
LIMITE_MUNICIPIOS_PROCESSADOS = 10
TX_POR_CLIENTE_ESPERADO = 15.0 # Ajustado para um valor mais realista de transações por cliente/mês
PROBABILIDADE_TRANSACAO_INTERMUNICIPAL = 0.20 

# === PARÂMETROS DE RISCO E CONTA (Para Geração de Clientes/Contas) ===
PROB_CONTA_ALTO_RISCO = 0.03 # Ajustado para refletir a raridade de contas de alto risco (3%)
PESO_CONTAS_POS_PIX = 0.70 # Probabilidade de a conta ter sido aberta após o lançamento do Pix (jan/2020)

# === PARÂMETROS DE FRAUDE (Bernoulli Condicional) ===
PROBABILIDADE_FRAUDE_BASE = 0.001 # Ajustado drasticamente para 0.1% (taxa base de fraude mais realista)
PROBABILIDADE_OUTLIER_BENIGNO = 0.04 # Taxa de outliers legítimos
MULTIPLICADOR_MAGNITUDE_OUTLIER = 25 # Mantido para garantir que outliers sejam detectáveis
MULTIPLICADOR_MAGNITUDE_FRAUDE = 50 # Mantido para garantir que a fraude seja de alta magnitude

# Parâmetros Específicos da Lógica de Fraude
PROBABILIDADE_FRAUDE_CONTA_DESTINO_RISCO = 0.60 
PROBABILIDADE_FRAUDE_CHAVE_RECENTE = 0.40      
DIAS_CHAVE_CONSIDERADA_RECENTE = 7             
MAX_DIAS_CADASTRO_CHAVE_RISCO = 5              

# === PROBABILIDADES DE TIPO DE FRAUDE (Distribuição Categórica) ===
PROBABILIDADES_TIPO_FRAUDE = {
    "valor_atipico": 0.30, 
    "engenharia_social": 0.25, 
    "tomada_de_conta": 0.15, 
    "triangulacao_conta_laranja": 0.15, 
    "fraude_qr_code": 0.10, 
    "ataque_de_frequencia": 0.05
}


## Inicialização do Ambiente e Esquemas

In [0]:

# INFO: Inicializando Spark Session e Faker

fake = Faker('pt_BR')
np.random.seed(42)
random.seed(42)

spark = SparkSession.builder.appName(NOME_APLICACAO_SPARK).getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", "200")

print("INFO: Definindo esquemas explícitos para UDFs e tabelas Delta...")

SCHEMA_CLIENTES_UDF = StructType([
    StructField("nome", StringType()),
    StructField("registro_nacional", StringType()),
    StructField("nascido_em", DateType())
])

SCHEMA_CONTAS_UDF = StructType([
    StructField("id", StringType()),
    StructField("saldo", DoubleType()),
    StructField("aberta_em", DateType()),
    StructField("agencia", StringType()),
    StructField("numero", StringType()),
    StructField("id_tipo_conta", IntegerType()),
    StructField("ispb_instituicao", StringType()),
    StructField("id_cliente", StringType())
])

SCHEMA_CHAVES_UDF = StructType([
    StructField("id", StringType()),
    StructField("chave", StringType()),
    StructField("id_tipo_chave", IntegerType()),
    StructField("cadastrada_em", DateType()),
    StructField("id_conta", StringType())
])

SCHEMA_TRANSACOES_UDF = StructType([
    StructField("id", StringType()),
    StructField("valor", DoubleType()),
    StructField("data", TimestampType()),
    StructField("mensagem", StringType()),
    StructField("id_conta_origem", StringType()),
    StructField("id_conta_destino", StringType()),
    StructField("id_tipo_iniciacao_pix", IntegerType()),
    StructField("id_finalidade_pix", IntegerType()),
    StructField("is_fraud", IntegerType()),
    StructField("fraud_type", StringType()),
    StructField("id_transacao_cadeia_pai", StringType())
])

SCHEMA_CLIENTES_FINAL = StructType([
    StructField("id", StringType(), False),
    StructField("nome", StringType()),
    StructField("id_natureza", IntegerType()),
    StructField("registro_nacional", StringType()),
    StructField("nascido_em", DateType()),
    StructField("estado_ibge", IntegerType()),
    StructField("municipio_ibge", IntegerType())
])

SCHEMA_CONTAS_FINAL = StructType(
    SCHEMA_CONTAS_UDF.fields + [
        StructField("is_high_risk", IntegerType()),
        StructField("estado_ibge", IntegerType()),
        StructField("municipio_ibge", IntegerType())
    ]
)

SCHEMA_CHAVES_PIX_FINAL = StructType(
    SCHEMA_CHAVES_UDF.fields + [
        StructField("estado_ibge", IntegerType()),
        StructField("municipio_ibge", IntegerType())
    ]
)

SCHEMA_TRANSACOES_FINAL = StructType(
    SCHEMA_TRANSACOES_UDF.fields + [
        StructField("estado_ibge", IntegerType())
    ]
)


# AGRUPAMENTO DE CONFIGURAÇÃO

config_geracao = {
    "PROBABILIDADE_FRAUDE_BASE": PROBABILIDADE_FRAUDE_BASE,
    "PROB_CONTA_ALTO_RISCO": PROB_CONTA_ALTO_RISCO,
    "PROBABILIDADE_FRAUDE_CONTA_DESTINO_RISCO": PROBABILIDADE_FRAUDE_CONTA_DESTINO_RISCO,
    "PROBABILIDADE_FRAUDE_CHAVE_RECENTE": PROBABILIDADE_FRAUDE_CHAVE_RECENTE,
    "DIAS_CHAVE_CONSIDERADA_RECENTE": DIAS_CHAVE_CONSIDERADA_RECENTE,
    "MAX_DIAS_CADASTRO_CHAVE_RISCO": MAX_DIAS_CADASTRO_CHAVE_RISCO,
    "MULTIPLICADOR_MAGNITUDE_OUTLIER": MULTIPLICADOR_MAGNITUDE_OUTLIER,
    "MULTIPLICADOR_MAGNITUDE_FRAUDE": MULTIPLICADOR_MAGNITUDE_FRAUDE,
    "PROBABILIDADES_TIPO_FRAUDE": PROBABILIDADES_TIPO_FRAUDE,
    "PESO_CONTAS_POS_PIX": PESO_CONTAS_POS_PIX
}


# 6. CARREGAMENTO DE DADOS AUXILIARES

print("INFO: Carregando dados de dimensão e estatísticas...")
try:
    instituicoes_pd = spark.table("transacoes_db.copper.instituicoes").toPandas()
    tipos_conta_pd = spark.table("transacoes_db.copper.tipos_conta").toPandas()
    perfis_de_uso_dict = spark.table("transacoes_db.pix_baseline_metricas.perfil_de_usuarios").toPandas().to_dict('records')
    LISTA_ISPBS_LOCAL = instituicoes_pd['ispb'].tolist()
    LISTA_TIPOS_CONTA_LOCAL = tipos_conta_pd['id'].tolist()
    print("INFO: Dados auxiliares e estatísticas carregados com sucesso.")
except Exception as e:
    print(f"ERRO: Falha ao carregar dados de dimensão ou estatísticas. Erro: {e}"); raise e

## Funções de Geração e Salvamento

As funções abaixo trabalham em conjunto para criar a população e as transações de um município:

---

### 1. `_gerar_clientes`, `_gerar_contas`, `_gerar_chaves_pix`

- **_gerar_clientes**  
  Cria o número especificado de Pessoas Físicas (PF) e Jurídicas (PJ), gerando IDs, nomes, CPFs/CNPJs e datas de nascimento/fundação.

- **_gerar_contas**  
  Para cada cliente, cria um número aleatório de contas bancárias.

- **_gerar_chaves_pix**  
  Para cada conta, gera uma chave PIX (CPF, e-mail, telefone, etc.), garantindo que a data de cadastro da chave seja sempre posterior à data de abertura da conta.

---

### 2. `_gerar_detalhes_transacao_python_vetorizado`

Função responsável por criar os detalhes de cada transação:

- **Data da Transação:**  
  Seleciona uma data e hora aleatória dentro do mês de referência.

- **Lógica de Fraude:**  
  Define a probabilidade de fraude com base em regras, como se a conta destino é de risco ou se a chave é recente.

- **Valor por Tipo de Conta:**  
  Gera um valor monetário condizente com o tipo de transação (ex: salário, transferência para poupança, etc.).

- **Multiplicadores de Fraude e Outlier:**  
  Aumenta drasticamente o valor da transação se ela for fraudulenta ou um outlier.

- **Fraudes em Cadeia:**  
  Para certos tipos de fraude, divide o valor inicial em várias subtransações menores, simulando lavagem de dinheiro.

---

### 3. `gerar_transacoes`

Orquestra a geração das transações para um município em um determinado mês:

- **Divisão do Volume:**  
  Separa o total de transações em locais e intermunicipais.

- **Geração de Pares Locais:**  
  Sorteia aleatoriamente pares de contas (origem e destino) do mesmo município.

- **Geração de Pares Intermunicipais:**  
  Sorteia contas de origem do município atual e contas de destino de outros municípios.

- **União e Enriquecimento:**  
  Junta os dois conjuntos de pares e busca as informações necessárias para gerar os detalhes de cada transação.

In [0]:
@F.pandas_udf(SCHEMA_CLIENTES_UDF)
def _gerar_detalhes_cliente_udf(id_natureza: pd.Series) -> pd.DataFrame:
    local_fake = Faker('pt_BR')
    resultados = []
    for natureza in id_natureza:
        if natureza == 1:
            nascido_em = local_fake.date_of_birth(minimum_age=18, maximum_age=80)
            resultados.append({
                "nome": local_fake.name(),
                "registro_nacional": local_fake.cpf(),
                "nascido_em": nascido_em
            })
        else:
            nascido_em = local_fake.date_between(start_date='-20y', end_date='-1y')
            resultados.append({
                "nome": local_fake.company(),
                "registro_nacional": local_fake.cnpj(),
                "nascido_em": nascido_em
            })
    return pd.DataFrame(resultados)

def _gerar_clientes(num_pf: int, num_pj: int, estado_ibge: int, municipio_ibge: int) -> DataFrame:
    df_base_pf = spark.range(num_pf).withColumn("id_natureza", F.lit(1))
    df_base_pj = spark.range(num_pj).withColumn("id_natureza", F.lit(2))
    df_base_clientes = df_base_pf.union(df_base_pj)
    return (
        df_base_clientes
        .withColumn("id", F.expr("uuid()"))
        .withColumn("detalhes", _gerar_detalhes_cliente_udf(F.col("id_natureza")))
        .select(
            "id",
            F.col("detalhes.nome").alias("nome"),
            "id_natureza",
            F.col("detalhes.registro_nacional").alias("registro_nacional"),
            F.col("detalhes.nascido_em").alias("nascido_em"),
            F.lit(estado_ibge).alias("estado_ibge"),
            F.lit(municipio_ibge).alias("municipio_ibge")
        )
    )

def _gerar_detalhes_conta_python(
    iterator: Iterator[pd.DataFrame], config: dict, tipos_conta: list, ispbs: list
) -> Iterator[pd.DataFrame]:
    local_fake = Faker('pt_BR')
    for lote in iterator:
        resultados = []
        for row in lote.itertuples(index=False):
            if row.id_natureza == 1:
                saldo = round(np.random.lognormal(mean=6, sigma=1.5), 2)
                tipo_conta = random.choice(tipos_conta)
            else:
                saldo = round(np.random.lognormal(mean=9, sigma=1.8), 2)
                tipo_conta = random.choice([c for c in tipos_conta if c in [1, 3]])
            if random.random() < config['PESO_CONTAS_POS_PIX']:
                aberta_em = local_fake.date_between(start_date='-3y', end_date='-1M')
            else:
                aberta_em = local_fake.date_between(start_date='-10y', end_date='-3y')
            resultados.append({
                "id": str(uuid.uuid4()),
                "saldo": saldo,
                "aberta_em": aberta_em,
                "agencia": local_fake.numerify('####'),
                "numero": local_fake.numerify('#####-#'),
                "id_tipo_conta": tipo_conta,
                "ispb_instituicao": random.choice(ispbs),
                "id_cliente": row.id_cliente
            })
        yield pd.DataFrame(resultados)

def _gerar_contas(df_clientes: DataFrame, estado_ibge: int, municipio_ibge: int) -> DataFrame:
    df_clientes_com_num_contas = df_clientes.withColumn(
        "num_contas",
        F.when(F.col("id_natureza") == 1, F.floor(F.rand() * 2) + 1)
        .otherwise(F.floor(F.rand() * 5) + 1)
    )
    df_contas_base = df_clientes_com_num_contas.select(
        F.col("id").alias("id_cliente"),
        "id_natureza",
        F.explode(F.sequence(F.lit(1), F.col("num_contas")))
    )
    gerador_com_contexto = functools.partial(
        _gerar_detalhes_conta_python,
        config=config_geracao,
        tipos_conta=LISTA_TIPOS_CONTA_LOCAL,
        ispbs=LISTA_ISPBS_LOCAL
    )
    return (
        df_contas_base
        .mapInPandas(gerador_com_contexto, schema=SCHEMA_CONTAS_UDF)
        .withColumn("estado_ibge", F.lit(estado_ibge))
        .withColumn("municipio_ibge", F.lit(municipio_ibge))
    )

def _gerar_detalhes_chave_udf(iterator: Iterator[pd.DataFrame], config: dict) -> Iterator[pd.DataFrame]:
    local_fake = Faker('pt_BR')
    for lote in iterator:
        resultados = []
        for row in lote.itertuples(index=False):
            try:
                data_abertura_obj = pd.to_datetime(row.aberta_em).date()
                if hasattr(row, 'is_high_risk') and row.is_high_risk == 1:
                    dias_para_cadastrar = random.randint(1, config['MAX_DIAS_CADASTRO_CHAVE_RISCO'])
                else:
                    dias_para_cadastrar = random.randint(1, 90)
                cadastrada_em = data_abertura_obj + timedelta(days=dias_para_cadastrar)
                if row.id_natureza == 1:
                    tipos_possiveis = {
                        1: row.registro_nacional,
                        2: local_fake.email(),
                        3: local_fake.phone_number(),
                        4: str(uuid.uuid4())
                    }
                else:
                    tipos_possiveis = {
                        5: row.registro_nacional,
                        2: local_fake.company_email(),
                        4: str(uuid.uuid4())
                    }
                tipo_chave = random.choice(list(tipos_possiveis.keys()))
                resultados.append({
                    "id": str(uuid.uuid4()),
                    "chave": tipos_possiveis[tipo_chave],
                    "id_tipo_chave": tipo_chave,
                    "cadastrada_em": cadastrada_em,
                    "id_conta": row.id_conta
                })
            except Exception as e:
                print(f"ERRO NA GERAÇÃO DE CHAVES: {e}, DADOS: {row}")
        if resultados:
            yield pd.DataFrame(resultados)

def _gerar_chaves_pix(
    df_contas_completo: DataFrame,
    df_clientes_completo: DataFrame,
    estado_ibge: int,
    municipio_ibge: int
) -> DataFrame:
    df_contas_com_cliente = (
        df_contas_completo
        .join(df_clientes_completo, df_contas_completo.id_cliente == df_clientes_completo.id, "inner")
        .select(
            df_contas_completo.id.alias("id_conta"),
            "id_natureza",
            "registro_nacional",
            "aberta_em",
            df_contas_completo.is_high_risk
        )
    )
    gerador_com_contexto = functools.partial(_gerar_detalhes_chave_udf, config=config_geracao)
    return (
        df_contas_com_cliente
        .mapInPandas(gerador_com_contexto, schema=SCHEMA_CHAVES_UDF)
        .withColumn("estado_ibge", F.lit(estado_ibge))
        .withColumn("municipio_ibge", F.lit(municipio_ibge))
    )

def _obter_params_tempo(ano: int, mes: int) -> tuple:
    # Simula a função original para obter o período de tempo
    import calendar
    from datetime import datetime
    num_dias = calendar.monthrange(ano, mes)[1]
    primeiro_dia = datetime(ano, mes, 1)
    ultimo_dia = datetime(ano, mes, num_dias, 23, 59, 59)
    return primeiro_dia, (ultimo_dia - primeiro_dia).total_seconds()

def _gerar_detalhes_transacao_python_vetorizado(
    iterator: Iterator[pd.DataFrame], ano: int, mes: int, config: dict, perfis_uso: list
) -> Iterator[pd.DataFrame]:
    """
    Função de User Defined Function (UDF) do Spark para Pandas que gera
    detalhes da transação, incluindo a lógica de fraude multi-camadas.
    """
    primeiro_dia, delta_segundos = _obter_params_tempo(ano, mes)
    local_fake = Faker('pt_BR') # Usado apenas para contexto, não na lógica principal

    for lote in iterator:
        n = len(lote)
        if n == 0:
            continue

        # --- LÓGICA VETORIZADA ORIGINAL (CALCULA is_fraud E valor) ---

        lote['data'] = primeiro_dia + pd.to_timedelta(np.random.uniform(0, delta_segundos, n), unit='s')
        
        # Lógica de Fraude
        lote['chave_destino_cadastrada_em'] = pd.to_datetime(lote['chave_destino_cadastrada_em'])
        delta_dias = (lote['data'] - lote['chave_destino_cadastrada_em']).dt.days
        lote['is_high_risk'] = lote['is_high_risk'].fillna(0).astype(int)
        
        # Simulação de PROBABILIDADE_FRAUDE_CONTA_DESTINO_RISCO, PROBABILIDADE_FRAUDE_CHAVE_RECENTE e PROBABILIDADE_FRAUDE_BASE
        # Valores placeholder para simular a lógica de fraude dinâmica (o código original usava valores de 'config')
        prob_fraude_risco = config.get('PROBABILIDADE_FRAUDE_CONTA_DESTINO_RISCO', 0.1)
        prob_fraude_recente = config.get('PROBABILIDADE_FRAUDE_CHAVE_RECENTE', 0.05)
        prob_fraude_base = config.get('PROBABILIDADE_FRAUDE_BASE', 0.001)
        dias_chave_recente = config.get('DIAS_CHAVE_CONSIDERADA_RECENTE', 7)

        prob_fraude_dinamica = np.select(
            [
                lote['is_high_risk'] == 1,
                (delta_dias >= 0) & (delta_dias <= dias_chave_recente)
            ],
            [
                prob_fraude_risco,
                prob_fraude_recente
            ],
            default=prob_fraude_base
        )
        rand_event = np.random.rand(n)
        lote['is_fraud'] = (rand_event < prob_fraude_dinamica).astype(int)

        # Lógica de Valor
        lote['id_tipo_conta_origem'] = lote['id_tipo_conta_origem'].fillna(0).astype(int)
        lote['id_tipo_conta_destino'] = lote['id_tipo_conta_destino'].fillna(0).astype(int)
        
        # Simulação de MULTIPLICADOR_MAGNITUDE_FRAUDE e MULTIPLICADOR_MAGNITUDE_OUTLIER
        multiplicador_fraude = config.get('MULTIPLICADOR_MAGNITUDE_FRAUDE', 5.0)
        multiplicador_outlier = config.get('MULTIPLICADOR_MAGNITUDE_OUTLIER', 2.5)

        mean_log_valor = np.log(150) # Valor base para simplificação
        valores_base = np.random.lognormal(mean=mean_log_valor, sigma=0.8, size=n)

        prob_outlier = 0.04
        is_outlier = (~(lote['is_fraud'] == 1)) & (np.random.rand(n) < prob_outlier)
        multiplicadores = np.select(
            [lote['is_fraud'] == 1, is_outlier],
            [multiplicador_fraude, multiplicador_outlier],
            default=1.0
        )
        lote['valor'] = np.maximum(0.01, valores_base * multiplicadores).round(2)

        # Seleção do Tipo de Fraude
        probs_tipo_fraude_keys = list(config.get('PROBABILIDADES_TIPO_FRAUDE', {"triangulacao_conta_laranja": 0.5, "ataque_de_frequencia": 0.5}).keys())
        probs_tipo_fraude_values = list(config.get('PROBABILIDADES_TIPO_FRAUDE', {"triangulacao_conta_laranja": 0.5, "ataque_de_frequencia": 0.5}).values())
        tipos_fraude = np.random.choice(probs_tipo_fraude_keys, n, p=probs_tipo_fraude_values)
        lote['fraud_type'] = np.where(lote['is_fraud'] == 1, tipos_fraude, None)
        
        # Adiciona colunas básicas
        lote['id'] = [str(uuid.uuid4()) for _ in range(n)]
        lote['mensagem'] = "Pagamento via Pix"
        lote['id_tipo_iniciacao_pix'] = np.random.randint(1, 4, n)
        lote['id_finalidade_pix'] = np.random.randint(1, 5, n)
        lote['id_transacao_cadeia_pai'] = None

        colunas_para_remover = [
            col for col in [
                'chave_destino_cadastrada_em',
                'is_high_risk',
                'id_tipo_conta_origem',
                'id_tipo_conta_destino'
            ] if col in lote.columns
        ]
        lote_final = lote.drop(columns=colunas_para_remover)

        # --- NOVA LÓGICA DE TRIANGULAÇÃO MULTI-CAMADAS (ROOT -> 3 -> 9) ---
        resultados_finais = []
        for row in lote_final.itertuples(index=False):
            if row.is_fraud and row.fraud_type in ["triangulacao_conta_laranja", "ataque_de_frequencia"]:
                # NÍVEL 1: Transação Raiz (Recebimento na Conta Laranja)
                id_fraude_raiz = row.id
                resultados_finais.append(row._asdict()) # Adiciona a transação raiz

                # Variáveis de Configuração para a Dispersão
                # Define um número aleatório de subdivisões por nível (ex: 2 a 5)
                min_subs = 2
                max_subs = 10
                
                num_subs_nivel2 = random.randint(min_subs, max_subs)
                
                intervalo_segundos_nivel2 = (1, 1800) # 1s a 30 minutos
                intervalo_segundos_nivel3 = (1801, 7200) # 30 min a 2 horas

                # NÍVEL 2: Dispersão da Conta Laranja (num_subs_nivel2 transações)
                valores_nivel2 = np.random.dirichlet(np.ones(num_subs_nivel2)) * row.valor
                
                for k in range(num_subs_nivel2):
                    id_transacao_nivel2 = str(uuid.uuid4())
                    id_conta_destino_nivel2 = str(uuid.uuid4()) # Nova Conta-Mula do Nível 2
                    segundos_offset_nivel2 = random.uniform(*intervalo_segundos_nivel2)
                    
                    transacao_nivel2 = {
                        "id": id_transacao_nivel2,
                        "valor": round(max(0.01, valores_nivel2[k]), 2),
                        "data": row.data + timedelta(seconds=segundos_offset_nivel2),
                        "mensagem": f"Dispersão N2 (Parte {k+1}/{num_subs_nivel2})",
                        "id_conta_origem": row.id_conta_destino, # Origem: Conta Laranja (Destino da Raiz)
                        "id_conta_destino": id_conta_destino_nivel2,
                        "id_tipo_iniciacao_pix": random.randint(1, 3),
                        "id_finalidade_pix": random.randint(1, 4),
                        "is_fraud": 1,
                        "fraud_type": row.fraud_type,
                        "id_transacao_cadeia_pai": id_fraude_raiz
                    }
                    resultados_finais.append(transacao_nivel2)

                    # NÍVEL 3: Dispersão das Contas do Nível 2 (num_subs_nivel3 sub-transações cada)
                    num_subs_nivel3 = random.randint(min_subs, max_subs)
                    valores_nivel3 = np.random.dirichlet(np.ones(num_subs_nivel3)) * transacao_nivel2["valor"]

                    for l in range(num_subs_nivel3):
                        segundos_offset_nivel3 = random.uniform(*intervalo_segundos_nivel3)
                        
                        transacao_nivel3 = {
                            "id": str(uuid.uuid4()),
                            "valor": round(max(0.01, valores_nivel3[l]), 2),
                            "data": transacao_nivel2["data"] + timedelta(seconds=segundos_offset_nivel3),
                            "mensagem": f"Dispersão N3 (Sub-parte {l+1}/{num_subs_nivel3})",
                            "id_conta_origem": id_conta_destino_nivel2, # Origem: Conta Destino do Nível 2
                            "id_conta_destino": str(uuid.uuid4()), # Conta-Mula Final
                            "id_tipo_iniciacao_pix": random.randint(1, 3),
                            "id_finalidade_pix": random.randint(1, 4),
                            "is_fraud": 1,
                            "fraud_type": row.fraud_type,
                            "id_transacao_cadeia_pai": id_transacao_nivel2 # Pai: Transação do Nível 2
                        }
                        resultados_finais.append(transacao_nivel3)
            else:
                resultados_finais.append(row._asdict())
        
        yield pd.DataFrame(resultados_finais)

def gerar_transacoes(
    df_contas: DataFrame,
    volume_total: int,
    estado_ibge: int,
    municipio_ibge: int,
    ano: int,
    mes: int
) -> DataFrame:
    print(f"INFO: Gerando {volume_total} transações para {municipio_ibge} ({mes}/{ano})...")

    # Prepara dados auxiliares para os joins
    df_chaves_pix = spark.table("transacoes_db.copper.chaves_pix")
    window_chaves = Window.partitionBy("id_conta").orderBy(F.col("cadastrada_em").desc())
    df_chaves_recentes_por_conta = (
        df_chaves_pix
        .withColumn("rank", F.rank().over(window_chaves))
        .filter(F.col("rank") == 1)
        .select("id_conta", F.col("cadastrada_em").alias("chave_destino_cadastrada_em"))
    )

    contas_locais = (
        df_contas
        .select("id", "is_high_risk", "id_tipo_conta")
        .withColumnRenamed("id", "id_conta")
    )

    # Separa o volume de transações em locais e intermunicipais
    volume_intermunicipal = int(volume_total * PROBABILIDADE_TRANSACAO_INTERMUNICIPAL)
    volume_local = volume_total - volume_intermunicipal

    # Gera pares LOCAIS
    df_pares_locais = spark.createDataFrame([], StructType([]))  # df vazio
    if volume_local > 0 and contas_locais.count() > 1:
        contas_ids_locais = contas_locais.select("id_conta")
        num_contas_locais = contas_ids_locais.count()
        df_pares_indices_locais = (
            spark.range(volume_local)
            .repartition(100)
            .withColumn("idx_origem", (F.rand() * num_contas_locais).cast("long"))
            .withColumn("idx_destino", (F.rand() * num_contas_locais).cast("long"))
            .filter("idx_origem != idx_destino")
        )
        contas_numeradas_locais = contas_ids_locais.withColumn(
            "idx", F.row_number().over(Window.orderBy("id_conta")) - 1
        )
        df_pares_locais = (
            df_pares_indices_locais
            .join(contas_numeradas_locais.alias("orig"), df_pares_indices_locais.idx_origem == F.col("orig.idx"))
            .join(contas_numeradas_locais.alias("dest"), df_pares_indices_locais.idx_destino == F.col("dest.idx"))
            .select(
                F.col("orig.id_conta").alias("id_conta_origem"),
                F.col("dest.id_conta").alias("id_conta_destino")
            )
        )

    # Gera pares INTERMUNICIPAIS
    df_pares_intermunicipais = spark.createDataFrame([], StructType([]))
    if volume_intermunicipal > 0:
        contas_origem = (
            contas_locais
            .select("id_conta")
            .orderBy(F.rand())
            .limit(volume_intermunicipal)
            .withColumn("row_id", F.monotonically_increasing_id())
        )
        contas_destino_externas = (
            spark.table("transacoes_db.copper.contas")
            .filter(F.col("municipio_ibge") != municipio_ibge)
            .select(F.col("id").alias("id_conta_destino"))
            .orderBy(F.rand())
            .limit(volume_intermunicipal)
            .withColumn("row_id", F.monotonically_increasing_id())
        )
        df_pares_intermunicipais = (
            contas_origem
            .join(contas_destino_externas, "row_id")
            .select(F.col("id_conta").alias("id_conta_origem"), "id_conta_destino")
        )

    # Une os dois conjuntos de pares
    df_pares_total = df_pares_locais.unionByName(df_pares_intermunicipais)
    if df_pares_total.isEmpty():
        print("AVISO: Nenhum par de transação foi gerado.")
        return None

    # Enriquece os pares com os dados necessários para a UDF
    df_pares_enriquecidos = (
        df_pares_total
        .join(contas_locais.alias("orig"), df_pares_total.id_conta_origem == F.col("orig.id_conta"), "left")
        .join(spark.table("transacoes_db.copper.contas").alias("dest"), df_pares_total.id_conta_destino == F.col("dest.id"), "left")
        .join(df_chaves_recentes_por_conta, df_pares_total.id_conta_destino == df_chaves_recentes_por_conta.id_conta, "left")
        .select(
            "id_conta_origem",
            "id_conta_destino",
            F.col("orig.id_tipo_conta").alias("id_tipo_conta_origem"),
            F.col("dest.id_tipo_conta").alias("id_tipo_conta_destino"),
            "chave_destino_cadastrada_em",
            F.col("dest.is_high_risk").alias("is_high_risk")
        )
    )

    gerador_com_contexto = functools.partial(
        _gerar_detalhes_transacao_python_vetorizado,
        ano=ano,
        mes=mes,
        config=config_geracao,
        perfis_uso=perfis_de_uso_dict
    )
    df_transacoes_bruto = df_pares_enriquecidos.mapInPandas(gerador_com_contexto, schema=SCHEMA_TRANSACOES_UDF)
    return df_transacoes_bruto.withColumn("estado_ibge", F.lit(estado_ibge))

def salvar_dataframe_em_delta(df: DataFrame, nome_tabela_completo: str, modo: str = "append"):
    if df is None or df.isEmpty():
        print(f"AVISO: DataFrame para a tabela '{nome_tabela_completo}' está vazio.")
        return
    try:
        print(f"INFO: Salvando dados na tabela Delta: {nome_tabela_completo} (modo: {modo})...")
        df.write.format("delta").mode(modo).saveAsTable(nome_tabela_completo)
        print(f"✅ SUCESSO: Dados salvos em {nome_tabela_completo}.")
    except Exception as e:
        print(f"❌ ERRO ao salvar '{nome_tabela_completo}': {e}")
        raise e

def gerar_e_salvar_populacao(num_pf: int, num_pj: int, estado_ibge: int, municipio_ibge: int) -> DataFrame:
    print(f"INFO: Gerando e materializando população para {municipio_ibge} (PF: {num_pf}, PJ: {num_pj})...")
    df_clientes_gerado = _gerar_clientes(num_pf, num_pj, estado_ibge, municipio_ibge)
    salvar_dataframe_em_delta(df_clientes_gerado, "transacoes_db.copper.clientes", modo="append")
    df_clientes_materializado = spark.table("transacoes_db.copper.clientes").filter(F.col("municipio_ibge") == municipio_ibge)
    df_contas_gerado = (
        _gerar_contas(df_clientes_materializado, estado_ibge, municipio_ibge)
        .withColumn("is_high_risk", F.when(F.rand() < PROB_CONTA_ALTO_RISCO, 1).otherwise(0))
    )
    salvar_dataframe_em_delta(df_contas_gerado, "transacoes_db.copper.contas", modo="append")
    df_contas_materializado = spark.table("transacoes_db.copper.contas").filter(F.col("municipio_ibge") == municipio_ibge)
    df_chaves_pix = _gerar_chaves_pix(df_contas_materializado, df_clientes_materializado, estado_ibge, municipio_ibge)
    salvar_dataframe_em_delta(df_chaves_pix, "transacoes_db.copper.chaves_pix", modo="append")
    print(f"INFO: População para o município {municipio_ibge} adicionada com sucesso.")
    return df_contas_materializado

def limpar_tabelas_de_destino():
    print("INFO: Apagando tabelas de destino para recriação...")
    for tabela in ["clientes", "contas", "chaves_pix", "transacoes"]:
        spark.sql(f"DROP TABLE IF EXISTS transacoes_db.copper.{tabela}")
    print("INFO: ✅ Limpeza concluída.")

def criar_tabelas_de_destino():
    print("INFO: Criando tabelas de destino com os schemas corretos...")
    tabelas = {
        "transacoes_db.copper.clientes": (SCHEMA_CLIENTES_FINAL, ["estado_ibge", "municipio_ibge"]),
        "transacoes_db.copper.contas": (SCHEMA_CONTAS_FINAL, ["estado_ibge", "municipio_ibge"]),
        "transacoes_db.copper.chaves_pix": (SCHEMA_CHAVES_PIX_FINAL, ["estado_ibge", "municipio_ibge"]),
        "transacoes_db.copper.transacoes": (SCHEMA_TRANSACOES_FINAL, ["estado_ibge"])
    }
    for nome, (schema, part_cols) in tabelas.items():
        spark.createDataFrame([], schema).write.format("delta").partitionBy(part_cols).mode("overwrite").saveAsTable(nome)
        print(f"INFO: Tabela '{nome}' criada com sucesso.")

# Modelagem Matemática para Simulação de Dados Sintéticos

---

## 1. Introdução

Este documento detalha a modelagem matemática e estatística utilizada para gerar uma população e um volume de transações sintéticas que refletem o comportamento e as características de um ambiente financeiro real, em conformidade com dados estatísticos regionais (perfis de uso) e regras de negócio.

A modelagem se divide em três etapas principais:

1. **Dimensionamento do volume**
2. **Dimensionamento da população**
3. **Modelagem comportamental para a geração de transações**

---

## 2. Dimensionamento do Volume de Transações

O volume total de transações a ser gerado para o município é baseado em dados estatísticos conhecidos (média mensal de transações) e ajustado ao escopo da simulação por meio de um fator de amostragem.

### 2.1. Fórmula do Volume Anual

O volume total anual de transações a gerar ($V_{\text{ano}}$) é calculado pela seguinte relação:

$$
V_{\text{ano}} = (\overline{T}_{\text{mensal}} \times 12) \times P_{\text{amostra}}
$$

| Termo                | Definição                                                                                  |
|----------------------|-------------------------------------------------------------------------------------------|
| $V_{\text{ano}}$     | Volume anual total de transações a ser gerado.                                            |
| $\overline{T}_{\text{mensal}}$ | Média estatística do número de transações de pagadores por mês no município.         |
| $P_{\text{amostra}}$ | Percentual da amostra desejada (`DATA_SAMPLE_PERCENTAGE`), fração do dado real a simular. |

---

## 3. Dimensionamento da População Sintética

Com o volume de transações definido, é necessário estimar o número de clientes que compõem a população sintética, garantindo que o volume transacional seja distribuído de forma crível.

### 3.1. Cálculo do Número Total de Clientes

O número total de clientes a serem gerados ($N_{\text{clientes}}$) é estimado a partir da média mensal de transações, ajustado por um fator de escala de população ($F_{\text{escala}}$) e o percentual da amostra:

$$
N_{\text{clientes}} = \left\lfloor \frac{\overline{T}_{\text{mensal}}}{F_{\text{escala}}} \times P_{\text{amostra}} \right\rfloor
$$

| Termo             | Definição                                                                                      |
|-------------------|-----------------------------------------------------------------------------------------------|
| $N_{\text{clientes}}$ | Número total de clientes (Pessoa Física e Jurídica) a gerar.                                  |
| $F_{\text{escala}}$   | Fator de escala para relacionar volume transacional com número de clientes (`POPULATION_SCALING_FACTOR`). |

### 3.2. Distribuição por Natureza (Pessoa Física e Jurídica)

O total de clientes é dividido em Pessoa Física ($N_{\text{PF}}$) e Pessoa Jurídica ($N_{\text{PJ}}$) com base na proporção observada de transações de Pessoa Física ($\%_{\text{PF}}$) no município:

$$
N_{\text{PF}} = \lfloor N_{\text{clientes}} \times \%_{\text{PF}} \rfloor
$$

$$
N_{\text{PJ}} = N_{\text{clientes}} - N_{\text{PF}}
$$

---

## 4. Modelagem Comportamental e Geração de Transações

Esta seção detalha os modelos estatísticos e as regras condicionais utilizadas para gerar as transações individuais (valores, datas e características de fraude).

### 4.1. Distribuição Proporcional por Perfil

Para garantir que a "mistura" de tipos de transação no município sintético reflita a região, o volume total é distribuído proporcionalmente aos perfis de uso definidos.

Primeiro, calcula-se o peso ($W_i$) de cada Perfil $i$:

$$
W_i = \frac{Q_i}{\sum_{j=1}^{P} Q_j}
$$

Onde $Q_i$ é a quantidade de transações do Perfil $i$ e $\sum_{j=1}^{P} Q_j$ é a soma das quantidades de todos os perfis ($P$).

Em seguida, calcula-se o volume de transações específicas para o Perfil $i$ ($V_i$):

$$
V_i = \lfloor V_{\text{ano}} \times W_i \rfloor
$$

---

### 4.2. Modelagem de Valores Monetários (Distribuição Log-Normal)

A **Distribuição Log-Normal** é empregada para modelar valores monetários (saldos e valores de transação) devido à sua assimetria e natureza não negativa, o que se alinha ao comportamento de dados financeiros.

Os parâmetros $\hat{\mu}$ (média do logaritmo) e $\hat{\sigma}$ (desvio padrão do logaritmo) são estimados a partir de estatísticas robustas (mediana e quartis) dos perfis para calibrar a geração:

$$
\hat{\mu} = \ln(\text{Mediana})
$$

$$
\hat{\sigma} = \frac{\ln(P_{75}) - \ln(P_{25})}{1.349}
$$

O valor final da transação ($X$) é gerado a partir desta distribuição, onde o valor base é multiplicado por fatores de ajuste para simular fraudes ou valores outliers.

---

### 4.3. Simulação de Eventos (Ensaio de Bernoulli e Lógica Condicional)

#### A. Ensaio de Bernoulli

A ocorrência de eventos discretos, como a marcação de uma transação como fraude, é modelada como um **Ensaio de Bernoulli**, onde a variável aleatória binária $X$ (sucesso/fracasso) é determinada por um número aleatório $U \sim \text{Uniforme}(0, 1)$ e uma probabilidade dinâmica $p$:

$$
X =
\begin{cases}
1, & \text{se } U < p \ (\text{sucesso/fraude}) \\
0, & \text{se } U \geq p \ (\text{falha/normal})
\end{cases}
$$

#### B. Lógica Condicional Vetorizada (`np.select`)

A probabilidade de fraude ($P_{\text{fraude}}$), que define o parâmetro $p$ do Ensaio de Bernoulli, é calculada em função de múltiplas regras de negócio e características da transação (vetorizada pelo `np.select`), como demonstrado no exemplo:

$$
P_{\text{fraude}} =
\begin{cases}
P_1, & \text{se a conta destino é de alto risco} \\
P_2, & \text{se } \Delta_{\text{dias}} \leq D_{\text{recente}} \text{ (chave recente)} \\
P_3, & \text{caso contrário}
\end{cases}
$$

---

### 4.4. Simulação de Cadeias de Lavagem (Distribuição de Dirichlet)

Em cenários de fraude complexa (e.g., triangulação), a **Distribuição de Dirichlet** é utilizada para simular a pulverização de um valor total em $N$ sub-transações aleatórias, garantindo que a soma das proporções seja igual a 1.

O vetor de proporções $\vec{x} = (x_1, \dots, x_N)$ é gerado com base em parâmetros de concentração $\alpha_i$ (tipicamente uniformes, $\alpha_i = 1$) e aplicado ao valor da transação raiz: $V_i = x_i \cdot V_{\text{raiz}}$

$$
(x_1, \dots, x_N) \sim \mathrm{Dirichlet}(\alpha_1, \dots, \alpha_N)
$$

$$
\sum_{i=1}^N x_i = 1
$$

O valor de cada sub-transação $V_i$ é dado por:

$$
V_i = x_i \cdot V_{\text{raiz}}
$$

---

## 5. Conclusão

A integração da **Distribuição Log-Normal**, do **Ensaio de Bernoulli**, da **Lógica Condicional Vetorizada** e da **Distribuição de Dirichlet** permite que o script de geração simule de forma eficaz um ecossistema financeiro robusto, com comportamentos e patamares de risco que se assemelham aos dados reais.

---

In [0]:
# -----------------------------------------------------------------------------
# 8. ORQUESTRAÇÃO E EXECUÇÃO PRINCIPAL
# -----------------------------------------------------------------------------
try:
    limpar_tabelas_de_destino()
    criar_tabelas_de_destino()
    print("=============================================================================")
    print(f"INFO: Iniciando processo de geração ANUAL (Ano: {ANO_ESTATISTICA})...")
    df_estatisticas_base = (
        spark.table("transacoes_db.pix_baseline_metricas.relacao_pagadores_recebedores")
        .filter(F.col("Ano") == ANO_ESTATISTICA)
    )
    df_volumes_anuais = (
        df_estatisticas_base.groupBy(
            "cod_ibge_municipio", "municipio_nome", "cod_ibge_estado"
        ).agg(
            F.sum("total_tx_pagador").alias("volume_pagador_anual"),
            F.sum("total_tx_recebedor").alias("volume_recebedor_anual"),
            F.sum("total_tx_pagador_pf").alias("total_pf_anual"),
            F.sum("total_tx_pagador_pj").alias("total_pj_anual"),
        )
    )
    df_ranks_anuais = (
        df_volumes_anuais.withColumn(
            "rank_pagador_anual",
            F.rank().over(Window.orderBy(F.col("volume_pagador_anual").desc())),
        ).withColumn(
            "rank_recebedor_anual",
            F.rank().over(Window.orderBy(F.col("volume_recebedor_anual").desc())),
        )
    )
    df_com_rank_final = df_ranks_anuais.withColumn(
        "rank_combinado_anual",
        F.col("rank_pagador_anual") + F.col("rank_recebedor_anual"),
    )
    municipios_a_processar_lista = (
        df_com_rank_final.orderBy(F.col("rank_combinado_anual").asc())
        .limit(LIMITE_MUNICIPIOS_PROCESSADOS)
        .collect()
    )
    total_municipios = len(municipios_a_processar_lista)
    print(f"INFO: {total_municipios} municípios selecionados para processar.")
    id_municipios_selecionados = [
        row["cod_ibge_municipio"] for row in municipios_a_processar_lista
    ]
    df_estatisticas_filtrado = df_estatisticas_base.filter(
        F.col("cod_ibge_municipio").isin(id_municipios_selecionados)
    )
    volume_total_base_original = (
        df_estatisticas_filtrado.agg(F.sum("total_tx_pagador")).first()[0] or 0
    )
    volume_total_base_escalado = int(volume_total_base_original * FATOR_ESCALA_VOLUME)
    if (
        volume_total_base_original > 0
        and volume_total_base_escalado > LIMITE_ABSOLUTO_TX
    ):
        fator_escala_final = LIMITE_ABSOLUTO_TX / volume_total_base_original
    else:
        fator_escala_final = FATOR_ESCALA_VOLUME
    print(f"INFO: Fator de escala final: {fator_escala_final:.8f}")

    for i, municipio_row in enumerate(municipios_a_processar_lista):
        codigo_municipio = municipio_row["cod_ibge_municipio"]
        codigo_estado = municipio_row["cod_ibge_estado"]
        nome_municipio = municipio_row["municipio_nome"]
        print(
            f"\n================== Processando Município {i+1}/{total_municipios}: {nome_municipio} ({codigo_municipio}) =================="
        )
        volume_pf_anual = int(municipio_row["total_pf_anual"] * FATOR_ESCALA_VOLUME)
        volume_pj_anual = int(municipio_row["total_pj_anual"] * FATOR_ESCALA_VOLUME)
        num_pf = max(1, int(volume_pf_anual / (TX_POR_CLIENTE_ESPERADO * 12)))
        num_pj = max(1, int(volume_pj_anual / (TX_POR_CLIENTE_ESPERADO * 12)))
        df_contas_do_municipio = gerar_e_salvar_populacao(
            num_pf=num_pf,
            num_pj=num_pj,
            estado_ibge=codigo_estado,
            municipio_ibge=codigo_municipio,
        )
        for mes in range(1, 13):
            print(
                f"\n--- Processando Mês {mes}/{ANO_ESTATISTICA} para {nome_municipio} ---"
            )
            stats_mensal_row = (
                df_estatisticas_filtrado.filter(
                    (F.col("cod_ibge_municipio") == codigo_municipio)
                    & (F.col("Mes") == mes)
                ).first()
            )
            if not stats_mensal_row:
                print(f"AVISO: Sem estatísticas para {mes}/{ANO_ESTATISTICA}. Pulando.")
                continue
            volume_total_original = stats_mensal_row["total_tx_pagador"]
            volume_total = int(volume_total_original * fator_escala_final)
            if volume_total == 0:
                print(
                    f"AVISO: Volume de transações base para {mes}/{ANO_ESTATISTICA} é 0. Pulando."
                )
                continue
            print(
                f"      Volume Original: {volume_total_original} | Volume BASE Alvo: {volume_total}"
            )
            try:
                df_transacoes = gerar_transacoes(
                    df_contas=df_contas_do_municipio,
                    volume_total=volume_total,
                    estado_ibge=codigo_estado,
                    municipio_ibge=codigo_municipio,
                    ano=ANO_ESTATISTICA,
                    mes=mes,
                )
                salvar_dataframe_em_delta(
                    df_transacoes, "transacoes_db.copper.transacoes", modo="append"
                )
            except Exception as e:
                print(f"ERRO CRÍTICO no mês {mes}/{ANO_ESTATISTICA}: {e}")
                import traceback

                traceback.print_exc()
finally:
    print("\nINFO: O script chegou ao fim.")

print("\n=============================================================================")
print("INFO: Processo de geração de dados sintéticos concluído.")
print("=============================================================================")